In [8]:
#imports
import pandas as pd
from helper import pickle_store, pickle_restore
import dateutil
from adem import *
from collections import Counter 


#fetch all rawdata and save as pickle format for speed.
dev_raw_df = pd.read_csv("r4.2/device.csv") # 2629979 rows, 11 columns
# Select small data till 1 April
pickle_store("pickle/dev_rawdf_file", dev_raw_df) #raw means not indexed by Timeindex

#fetch all rawdata and save as pickle format for speed.
log_raw_df = pd.read_csv("r4.2/logon.csv") # 2629979 rows, 11 columns
# Select small data till 1 April
pickle_store("pickle/log_rawdf_file", log_raw_df) #raw means not indexed by Timeindex



In [9]:
isdev=True  #f4 true, f7false 

#get email data of first 3 months: data is without date column converted to timedates type
if isdev:
    dev_raw_df = pickle_restore("pickle/dev_rawdf_file")
else:
    dev_raw_df = pickle_restore("pickle/log_rawdf_file")

dev_act_feature_df=dev_raw_df[['user', 'date', 'pc', 'activity']]


dev_act_feature_df['date'] = pd.to_datetime(dev_act_feature_df['date'])
# mask = dev_act_feature_df['date'] <= end_d
mask = (dev_act_feature_df['date'] <= end_d) & (dev_act_feature_df['date'] >= start_d)
dev_act_feature_df = dev_act_feature_df.loc[mask]





#create a copy of orig df so that I can separately process for ubp pbp and cbp
dev_act_feature_cmnty_df = dev_act_feature_df.copy()
dev_act_feature_peer_df = dev_act_feature_df.copy()

# 1. in email_freq_feature_df add community and peer column with id as values.
usr_cmnty_map = pickle_restore("pickle/community_louvian_file")
dev_act_feature_cmnty_df['cmnty']=dev_act_feature_cmnty_df.apply(lambda row: usr_cmnty_map[row.user], axis=1)

usr_peer_map = pickle_restore("pickle/eid_role_map_file")
dev_act_feature_peer_df['peer']=dev_act_feature_peer_df.apply(lambda row: usr_peer_map[row.user], axis=1)


In [10]:
dev_act_feature_df

,user,date,pc,activity
131068,EIS0041,2010-06-01 00:47:01,PC-3124,Connect
131069,BSS0369,2010-06-01 00:47:38,PC-8508,Connect
131070,BSS0369,2010-06-01 00:47:58,PC-8508,Disconnect
131071,EIS0041,2010-06-01 00:51:34,PC-3124,Disconnect
131072,MOS0047,2010-06-01 01:23:00,PC-6688,Connect
...,...,...,...,...
305590,FMG0527,2010-12-30 22:41:33,PC-4256,Disconnect
305591,FMG0527,2010-12-30 23:08:43,PC-4256,Connect
305592,FMG0527,2010-12-30 23:12:54,PC-4256,Disconnect
305593,FMG0527,2010-12-30 23:34:33,PC-4256,Connect


In [ ]:
# #select data only till 1 April (4/1/2010). 

# # convert date column in date form to comapre and select small data till desired date.
# # df = dev_act_feature_df.copy()
# dev_act_feature_df['date'] = pd.to_datetime(dev_act_feature_df['date'])
# mask = dev_act_feature_df['date'] <= '04-2-2010'
# dev_act_feature_df = dev_act_feature_df.loc[mask]


# dev_act_feature_cmnty_df['date'] = pd.to_datetime(dev_act_feature_cmnty_df['date'])
# mask = dev_act_feature_cmnty_df['date'] <= '04-2-2010'
# dev_act_feature_cmnty_df = dev_act_feature_cmnty_df.loc[mask]


# dev_act_feature_peer_df['date'] = pd.to_datetime(dev_act_feature_peer_df['date'])
# mask = dev_act_feature_peer_df['date'] <= '04-2-2010'
# dev_act_feature_peer_df = dev_act_feature_peer_df.loc[mask]

In [11]:
# Also NOTE that unlike emails I have converted the dates to datetime format. 
# Convert date back to string format
dev_act_feature_df['date']=dev_act_feature_df['date'].astype(str)
dev_act_feature_cmnty_df['date']=dev_act_feature_cmnty_df['date'].astype(str)
dev_act_feature_peer_df['date']=dev_act_feature_peer_df['date'].astype(str)

# dev_act_feature_df.info()

In [12]:
# Now data is ready to generate prep_dic using groupby, UBP. Feature = num_activity
def prep_ubp_dic(df):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=False) #NOTE converts day to day first year-dd-mm
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    #creates user, date wise groups and each internal group is a dataframe.
    # grp=tmp_email_freq_feature_df.groupby(['user', 'date'])

    # create user, date wise groups and count unique dates then add a separate column for the counts
    df=df.groupby(['user', 'date']).activity.agg('count').to_frame('num_usage').reset_index()

    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['user'])

    email_freq_feature_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        email_freq_feature_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()

    return email_freq_feature_dic


def prep_cbp_dic(df):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=False)
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    # create user, date wise groups and count unique dates then add a separate column for the counts
    df=df.groupby(['cmnty', 'date']).activity.agg('count').to_frame('num_usage').reset_index()

    cmnty_len = Counter(usr_cmnty_map.values())
    #make values of email freq avg of community
    df['num_usage'] = df.apply( lambda x: x.num_usage/cmnty_len[x.cmnty], axis=1)



    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['cmnty'])

    feature_cmnty_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        feature_cmnty_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()



    return feature_cmnty_dic

def prep_pbp_dic(df):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=False)
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    # create user, date wise groups and count unique dates then add a separate column for the counts
    df=df.groupby(['peer', 'date']).activity.agg('count').to_frame('num_usage').reset_index()

    cmnty_len = Counter(usr_peer_map.values())
    #make values of email freq avg of community
    df['num_usage'] = df.apply( lambda x: x.num_usage/cmnty_len[x.peer], axis=1)

    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['peer'])

    feature_peer_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        feature_peer_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()
    return feature_peer_dic



In [13]:
ubp_dev_act_feature_df = dev_act_feature_df.copy()
ubp_dev_act_feature_dic = prep_ubp_dic(ubp_dev_act_feature_df)


cbp_dev_act_feature_df = dev_act_feature_cmnty_df.copy()
cbp_dev_act_feature_dic = prep_cbp_dic(cbp_dev_act_feature_df)


pbp_dev_act_feature_df = dev_act_feature_peer_df.copy()
pbp_dev_act_feature_dic = prep_pbp_dic(pbp_dev_act_feature_df)


#store all preped dic
if isdev:
    pickle_store("pickle/f4/f4_ubp_file", ubp_dev_act_feature_dic)
    pickle_store("pickle/f4/f4_pbp_file", pbp_dev_act_feature_dic)
    pickle_store("pickle/f4/f4_cbp_file", cbp_dev_act_feature_dic)
else:
    pickle_store("pickle/f7/f7_ubp_file", ubp_dev_act_feature_dic)
    pickle_store("pickle/f7/f7_pbp_file", pbp_dev_act_feature_dic)
    pickle_store("pickle/f7/f7_cbp_file", cbp_dev_act_feature_dic)





In [ ]:
if isdev:
    ubp_dev_act_feature_dic = pickle_restore("pickle/f4/f4_ubp_file") 
    pbp_dev_act_feature_dic = pickle_restore("pickle/f4/f4_pbp_file") 
    cbp_dev_act_feature_dic = pickle_restore("pickle/f4/f4_cbp_file") 
else:
    ubp_dev_act_feature_dic = pickle_restore("pickle/f7/f7_ubp_file") 
    pbp_dev_act_feature_dic = pickle_restore("pickle/f7/f7_pbp_file") 
    cbp_dev_act_feature_dic = pickle_restore("pickle/f7/f7_cbp_file") 




In [ ]:
count = 0
feature_name = 'num_usage'
print('PBP')
pbp_events_dic={}
for key in ubp_dev_act_feature_dic:
    usr=key #'AAM0658'
    debug_df = ubp_dev_act_feature_dic[usr]
    debug_peer_df = pbp_dev_act_feature_dic[usr_peer_map[usr]]

    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    df_peer = debug_peer_df.copy()
    
    uname = key
    print(key)
    anom_ev = anom_calc_pbp(df, df_peer, feature_name, uname, ws=10, sig=3)

    if len(anom_ev['anomalies_dict']):
        pbp_events_dic[key]=anom_ev
    
    # count += 1
    # if count%1 == 0:
    #     break

print('CBP')
cbp_events_dic={}
for key in ubp_dev_act_feature_dic:
    usr=key
    debug_df = ubp_dev_act_feature_dic[usr]
    debug_cmnty_df = cbp_dev_act_feature_dic[usr_cmnty_map[usr]]

    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    df_cmnty = debug_cmnty_df.copy()

    uname = key
    print(key)
    anom_ev = anom_calc_cbp(df, df_cmnty, feature_name, uname, ws=10, sig=3)

    if len(anom_ev['anomalies_dict']):
        cbp_events_dic[key]=anom_ev
    
    # count += 1
    # if count%1 == 0:
    #     break


print('UBP')
ubp_events_dic={}
# email_freq_feature_dic.keys()
# Feature data format:  user: df [user, date, featurecolmn]
for key in ubp_dev_act_feature_dic:
    usr=key
    debug_df = ubp_dev_act_feature_dic[usr]


    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    print(key)
    anom_ev = anom_calc_ubp(df, feature_name, usr, ws=10, sig=3)

    if len(anom_ev['anomalies_dict']):
        ubp_events_dic[key]=anom_ev

    # count += 1
    # if count%100 == 0:
    #     break

In [ ]:
from helper import get_anom_list

anom_user_lst=[]
# for anom_usr in ubp_events_dic:
#     a_ev = ubp_events_dic[anom_usr]
#     print('dict a_ev=', a_ev['anomalies_dict'])
#     for k, v in a_ev['anomalies_dict'].items():
#         if v != 0:
#             f1_anom_user_lst.append(anom_usr)
#             break

anom_user_lst += get_anom_list(ubp_events_dic)
anom_user_lst += get_anom_list(pbp_events_dic)
anom_user_lst += get_anom_list(cbp_events_dic)

len(set(anom_user_lst))

# save list of anom users
if isdev:
    pickle_store("pickle/f4/f4_anom_set_file", set(anom_user_lst)) #raw means not indexed by Timeindex
else:
    pickle_store("pickle/f7/f7_anom_set_file", set(anom_user_lst)) #raw means not indexed by Timeindex


In [ ]:
len(set(anom_user_lst))

set(anom_user_lst)